### Richardson-Lucy Deconvolution with RedLionFish

For more information, please see https://github.com/rosalindfranklininstitute/RedLionfish

#### Deconvolved thumbnails

In [ ]:
def get_central_xy(img_filename):
    if img_filename.endswith(".mrc"):
        with mrcfile.open(img_filename) as mrc:
            img_data = mrc.data
    else:
        raise IOError(f"Please use an mrc instead of {os.path.splitext(img_filename)[-1]}")

    # Find central slices
    z_central = int(img_data.shape[0]/2)

    # xy, yz, xz image data
    xy_data = img_data[z_central, :, :]

    return xy_data


def show_xy_compare(xy1, xy2, title=None, labels=None):
    """ compare two xy-slices """        
    plt.figure(figsize=(10,5))
    plt.suptitle(title)

    for i, data in enumerate([xy1, xy2]):
        plt.subplot(1,2,i+1)
        plt.imshow(
            data,
            cmap="Greys_r",
            # vmin=np.percentile(data[data>0].flatten(), 15),
            # vmax=np.percentile(data[data>0].flatten(), 85)
        )
        plt.title(labels[i])
    
    plt.tight_layout()


def get_tomogram_filenames():
    # Get RLF mdout
    try:
        rlf_mdout = f"{os.getcwd()}/{proj_name}_rlf_deconv_mdout.yaml"
    except FileNotFoundError:
        print(f"{proj_name}_savurecon_mdout.yaml not found, cannot find tomogram filenames.")

    tomo_filenames = {}

    with open(rlf_mdout, "r") as f:
        output = yaml.load(f.read(), Loader=yaml.FullLoader)
        tomo_filenames["raw"] = output["raw_files"]
        tomo_filenames["deconvolved"] = output["outputs"]

    return tomo_filenames

tomogram_filenames = get_tomogram_filenames()

for tomogram in list(tomogram_filenames["raw"].keys()):
    xy_raw = get_central_xy(tomogram_filenames["raw"][tomogram])
    xy_deconv = get_central_xy(tomogram_filenames["deconvolved"][tomogram])
    show_xy_compare(
        xy_raw,
        xy_deconv,
        title=os.path.basename(tomogram_filenames["raw"][tomogram]),
        labels=["Original", "Deconvolved"]
    )